<a href="https://colab.research.google.com/github/liviasouza01/LLM_Challenge/blob/main/NM_LLM25_AnaliseSentimentosBagOfWords.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Processo Seletivo Engenheiro de LLM


versão 19 de janeiro de 2025

### Nome: Livia Souza

### E-mail: lsa5@cin.ufpe.br

## Instalação e importação de pacotes

In [1]:
!pip install datasets -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 16.8 MB/s eta 0:00:00


In [2]:
import torch
import random
from torch.utils.data import Dataset, DataLoader, random_split

from collections import Counter
import torch.nn as nn
import torch.optim as optim
from datasets import load_dataset

from sklearn.model_selection import train_test_split
import re
from tqdm import tqdm
from transformers import AutoTokenizer

## I - Vocabulário e Tokenização

### Exemplo do dataset

In [3]:
train_dataset = load_dataset("stanfordnlp/imdb", split="train")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.81k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [4]:
len(train_dataset)

25000

In [5]:
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'<br\s*/?>', ' ', text)
    text = re.sub(r'[^a-z\s\']', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    text = re.sub(r"\'s", " is", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"\'m", " am", text)
    text = re.sub(r"\'d", " would", text)
    text = re.sub(r"\'ll", " will", text)
    text = re.sub(r"\'t", " not", text)
    text = re.sub(r"\'ve", " have", text)
    return text

In [13]:
# limit the vocabulary size to 20000 most frequent tokens
vocab_size = 5000

counter = Counter()
for sample in tqdm(train_dataset, desc="Building vocabulary"):
    counter.update(preprocess_text(sample["text"]).split())

# create a vocabulary of the 20000 most frequent tokens
most_frequent_words = sorted(counter, key=counter.get, reverse=True)[:vocab_size]
vocab = {word: i for i, word in enumerate(most_frequent_words, 1)}
vocab_size = len(vocab)

Building vocabulary: 100%|██████████| 25000/25000 [00:05<00:00, 4896.23it/s]


## II - Dataset

In [20]:
class IMDBDataset(Dataset):
    def __init__(self, data, vocab):
        self.data = data
        self.vocab = vocab

        #Pre-process all data at once
        texts = [preprocess_text(sample["text"]) for sample in data]
        labels = [1 if sample["label"] == 1 else 0 for sample in data]

        #Vectorize all texts at once
        self.processed_data = []
        for text, label in zip(texts, labels):
            encoded = torch.zeros(len(self.vocab) + 1)
            for word in text.split():
                word_idx = self.vocab.get(word, 0)
                encoded[word_idx] = 1
            self.processed_data.append((encoded, torch.tensor(label)))

    def __len__(self):
        return len(self.processed_data)

    def __getitem__(self, idx):
        return self.processed_data[idx]

# Divisão treino/validação
train_size = int(0.8 * len(train_dataset))
val_size = len(train_dataset) - train_size
train_subset, val_subset = torch.utils.data.random_split(
    train_dataset, [train_size, val_size]
)

# Criação dos datasets
train_data = IMDBDataset(train_subset, vocab)
val_data = IMDBDataset(val_subset, vocab)
test_data = IMDBDataset(load_dataset("stanfordnlp/imdb", split="test"), vocab)

## III - Data Loader

In [21]:
batch_size = 128
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True,
                         num_workers=2, pin_memory=True)
val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=False,
                       num_workers=2, pin_memory=True)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False,
                        num_workers=2, pin_memory=True)

## IV - Modelo

In [22]:
class OneHotMLP(nn.Module):
    def __init__(self, vocab_size):
        super(OneHotMLP, self).__init__()

        self.fc1 = nn.Linear(vocab_size+1, 200)
        self.fc2 = nn.Linear(200, 1)

        self.relu = nn.ReLU()

    def forward(self, x):
        o = self.fc1(x.float())
        o = self.relu(o)
        return self.fc2(o)

# Model instantiation
model = OneHotMLP(vocab_size)

## V - Laço de Treinamento - Otimização da função de Perda pelo Gradiente descendente

In [17]:
# Verifica se há uma GPU disponível e define o dispositivo para GPU se possível,
# caso contrário, usa a CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
if device.type == 'cuda':
    print('GPU:', torch.cuda.get_device_name(torch.cuda.current_device()))
else:
    print('using CPU')

GPU: Tesla T4


In [23]:
import time

model = model.to(device)
# Define loss and optimizer
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.SGD(model.parameters(), lr=0.05)

num_epochs = 5

for epoch in range(num_epochs):
    start_time = time.time()

    model.train()
    total_train_loss = 0
    num_train_batches = 0

    for inputs, targets in train_loader:
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs.squeeze(), targets.float())
        loss.backward()
        optimizer.step()

        total_train_loss += loss.item()
        num_train_batches += 1


    # Validation phase - moved outside of the training loop
    model.eval()
    total_val_loss = 0
    num_val_batches = 0

    with torch.no_grad():
        for inputs, targets in val_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            loss = criterion(outputs.squeeze(), targets.float())
            total_val_loss += loss.item()
            num_val_batches += 1

    # Calculate average losses
    avg_train_loss = total_train_loss / num_train_batches
    avg_val_loss = total_val_loss / num_val_batches
    epoch_time = time.time() - start_time

    print(f'Epoch [{epoch+1}/{num_epochs}], '
          f'Train Loss: {avg_train_loss:.4f}, '
          f'Val Loss: {avg_val_loss:.4f}, '
          f'Elapsed Time: {epoch_time:.2f}s')

Epoch [1/5], Train Loss: 0.6455, Val Loss: 0.5649, Elapsed Time: 1.67s
Epoch [2/5], Train Loss: 0.4663, Val Loss: 0.4396, Elapsed Time: 1.61s
Epoch [3/5], Train Loss: 0.3657, Val Loss: 0.4199, Elapsed Time: 1.61s
Epoch [4/5], Train Loss: 0.3212, Val Loss: 0.3343, Elapsed Time: 2.13s
Epoch [5/5], Train Loss: 0.2963, Val Loss: 0.3260, Elapsed Time: 2.04s


## VI - Avaliação

In [24]:
## evaluation
model.eval()

with torch.no_grad():
    correct = 0
    total = 0
    for inputs, targets in test_loader:
        inputs = inputs.to(device)
        targets = targets.to(device)
        logits = model(inputs)
        predicted = torch.round(torch.sigmoid(logits.squeeze()))
        total += targets.size(0)
        correct += (predicted == targets).sum().item()

    print(f'Test Accuracy: {100 * correct / total}%')

Test Accuracy: 86.308%
